# Expresso Churn Checkpoint
This notebook does data import + exploration + profiling + cleaning + training/testing + model export.
I will then download the exported model and use it in Streamlit locally (PyCharm part).

**Outputs:**
- `profiling_report.html`
- `expresso_churn_model.joblib`


In [2]:
# 1) Install packages
!pip -q install ydata-profiling scikit-learn pandas numpy joblib matplotlib

In [3]:
# 2) Import libraries
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

import joblib

In [4]:
# 3) Load data
DATA_PATH = "/content/Expresso_churn_dataset.csv"
df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
print("Duplicates:", df.duplicated().sum())
df.head()

Shape: (1048575, 19)
Duplicates: 0


,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,NO,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,NO,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,NO,11,Mixt 250F=Unlimited_call24H,2.0,0


In [5]:
# 4) General info + basic stats
df.info()
display(df.describe(include="all").T.head(40))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 19 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   user_id         1048575 non-null  object 
 1   REGION          635141 non-null   object 
 2   TENURE          1048575 non-null  object 
 3   MONTANT         679931 non-null   float64
 4   FREQUENCE_RECH  679931 non-null   float64
 5   REVENUE         694797 non-null   float64
 6   ARPU_SEGMENT    694797 non-null   float64
 7   FREQUENCE       694797 non-null   float64
 8   DATA_VOLUME     532041 non-null   float64
 9   ON_NET          665211 non-null   float64
 10  ORANGE          612535 non-null   float64
 11  TIGO            420459 non-null   float64
 12  ZONE1           82630 non-null    float64
 13  ZONE2           66343 non-null    float64
 14  MRG             1048575 non-null  object 
 15  REGULARITY      1048575 non-null  int64  
 16  TOP_PACK        609188 non-null   ob

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
user_id,1048575,1048575,7cb5c71bf4ef411d5b7c6b3b7827193facb10a5d,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
REGION,635141,14,DAKAR,249806,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TENURE,1048575,8,K > 24 month,994587,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MONTANT,679931.0,NaN,NaN,NaN,5538.217109,7147.71261,10.0,1000.0,3000.0,7350.0,470000.0
FREQUENCE_RECH,679931.0,NaN,NaN,NaN,11.541451,13.282298,1.0,2.0,7.0,16.0,133.0
REVENUE,694797.0,NaN,NaN,NaN,5514.798879,7199.856252,1.0,1000.0,3000.0,7387.0,532177.0
ARPU_SEGMENT,694797.0,NaN,NaN,NaN,1838.272454,2399.947652,0.0,333.0,1000.0,2462.0,177392.0
FREQUENCE,694797.0,NaN,NaN,NaN,13.990561,14.698701,1.0,3.0,9.0,20.0,91.0
DATA_VOLUME,532041.0,NaN,NaN,NaN,3367.127268,13227.148631,0.0,0.0,260.0,2888.0,1556829.0
ON_NET,665211.0,NaN,NaN,NaN,277.695893,873.530717,0.0,5.0,27.0,157.0,50809.0


In [6]:
# 5) Profiling report
sample = df.sample(20000, random_state=42) if len(df) > 20000 else df
report = ProfileReport(sample, minimal=True, title="Expresso Churn Profiling Report")
report.to_file("profiling_report.html")
print("Saved: profiling_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 19/19 [00:01<00:00, 13.04it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Saved: profiling_report.html


In [7]:
# 6) Cleaning: remove duplicates, split target, drop obvious ID if present
df = df.drop_duplicates()

TARGET = "CHURN"
if "user_id" in df.columns:
    df = df.drop(columns=["user_id"])

X = df.drop(columns=[TARGET])
y = df[TARGET]

cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
num_cols = X.select_dtypes(exclude=["object"]).columns.tolist()

print("Categorical:", len(cat_cols), cat_cols[:10])
print("Numeric:", len(num_cols), num_cols[:10])

Categorical: 4 ['REGION', 'TENURE', 'MRG', 'TOP_PACK']
Numeric: 13 ['MONTANT', 'FREQUENCE_RECH', 'REVENUE', 'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO', 'ZONE1']


In [8]:
# 7) Handle outliers (IQR clipping on numeric)
def iqr_clip_df(df_num):
    df_num = df_num.copy()
    for c in df_num.columns:
        q1, q3 = df_num[c].quantile(0.25), df_num[c].quantile(0.75)
        iqr = q3 - q1
        df_num[c] = df_num[c].clip(q1 - 1.5*iqr, q3 + 1.5*iqr)
    return df_num

X[num_cols] = X[num_cols].apply(pd.to_numeric, errors="coerce")
X[num_cols] = iqr_clip_df(X[num_cols])

In [9]:
# 8) Preprocessing (missing values + encoding)
preprocess = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", Pipeline(steps=[
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("oh", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols)
    ]
)

In [13]:
# 9) Train/test classifier
# Drop rows where the target variable 'y' is NaN
df_cleaned = df.dropna(subset=[TARGET])

X_cleaned = df_cleaned.drop(columns=[TARGET])
y_cleaned = df_cleaned[TARGET]

# Ensure numeric columns are handled
X_cleaned[num_cols] = X_cleaned[num_cols].apply(pd.to_numeric, errors="coerce")
X_cleaned[num_cols] = iqr_clip_df(X_cleaned[num_cols])

X_train, X_test, y_train, y_test = train_test_split(
    X_cleaned, y_cleaned, test_size=0.2, random_state=42, stratify=y_cleaned
)

from sklearn.linear_model import SGDClassifier

model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", SGDClassifier(loss="log_loss", max_iter=1000, tol=1e-3))
])

model.fit(X_train, y_train)
pred = model.predict(X_test)

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.94      0.87      0.90    170355
           1       0.57      0.77      0.66     39360

    accuracy                           0.85    209715
   macro avg       0.76      0.82      0.78    209715
weighted avg       0.87      0.85      0.86    209715



In [16]:
# 10) Export model
joblib.dump(model, "expresso_churn_model.joblib")
print("Saved: expresso_churn_model.joblib")

Saved: expresso_churn_model.joblib


In [17]:
# 11) Download outputs to your PC
from google.colab import files
files.download("expresso_churn_model.joblib")
files.download("profiling_report.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>